In [5]:
import sys
sys.path.append("../../src")
import os
import datetime
import pandas as pd
import numpy as np
from example_lorenz import get_lorenz_data
from sindy_utils import library_size
from training import train_network
import tensorflow as tf

# Generate data

In [6]:
# generate training, validation, testing data
noise_strength = 1e-6
training_data = get_lorenz_data(1024, noise_strength=noise_strength)
validation_data = get_lorenz_data(20, noise_strength=noise_strength)

# Set up model and training parameters

In [7]:
params = {}

params['input_dim'] = 128
params['latent_dim'] = 3
params['model_order'] = 1
params['poly_order'] = 3
params['include_sine'] = False
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_z'] = 0.0
params['loss_weight_sindy_x'] = 1e-4
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
print(training_data['x'].shape[0])
params['batch_size'] = 1024
params['learning_rate'] = 1e-3

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 1

# training time cutoffs
params['max_epochs'] = 5001
params['refinement_epochs'] = 1001

256000


# Run training experiments

In [8]:
num_experiments = 1
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'lorenz_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
TRAINING
Epoch 0
   training loss 0.055861566215753555, (0.054228835, 1598.4082, 16.254604, 0.7272231)
   validation loss 0.04030901566147804, (0.038666036, 1803.2072, 16.357077, 0.7272231)
decoder loss ratio: 0.202899, decoder SINDy loss  ratio: 1.244901
Epoch 1
   training loss 0.04706043750047684, (0.045581795, 189.15039, 14.728531, 0.5788514)
   validation loss 0.032366104423999786, (0.030966012, 202.4324, 13.943036, 0.5788514)
decoder loss ratio: 0.162493, decoder SINDy loss  ratio: 1.061174
Epoch 2
   training loss 0.026412704959511757, (0.02503476, 139.51094, 13.730327, 0.49129364)
   validation loss 0.0190883819013834, (0.017741453, 141.95781, 13.42015, 0.49129364)
decoder loss ratio: 0.093098, decoder SINDy loss  ratio: 1.021378
Epoch 3
   training loss 0.01424498576670885, (0.013033007, 80.96227, 12.072458, 0.47325712)
   validation loss 0.012188449501991272, (0.011020565, 80.18498, 11.631509, 0.47325712)
decoder loss ratio: 0.057830, decoder SINDy loss  ratio: 0

KeyboardInterrupt: 